In [1]:
import os 
from langchain.chains import RetrievalQA
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain_experimental.agents.agent_toolkits.csv.base import create_csv_agent
from langchain.agents.agent_types import AgentType
from langchain.memory import ConversationBufferMemory
import tiktoken


In [3]:
os.environ['OPENAI_API_KEY'] = os.environ.get('OPENAI_API_KEY')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')
llm_model = "gpt-3.5-turbo"


In [4]:

txt_file_path = 'scalexi.txt'
loader = TextLoader(file_path=txt_file_path, encoding="utf-8")
data = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
data = text_splitter.split_documents(data)

In [8]:
# Create vector store
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(data, embedding=embeddings)

c:\Users\shana\.conda\envs\langchain_env\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [10]:
# Create conversation chain
llm = ChatOpenAI(temperature=0.7, model_name="gpt-4")
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        chain_type="stuff",
        retriever=vectorstore.as_retriever(),
        memory=memory
        )

c:\Users\shana\.conda\envs\langchain_env\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [11]:
query = "What is ScaleX Innovation?"
result = conversation_chain({"question": query})
answer = result["answer"]
answer

c:\Users\shana\.conda\envs\langchain_env\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'ScaleX Innovation is a pioneering company specializing in Generative AI and Large Language Models. They focus on integrating these transformative technologies into business strategies to enhance innovation and operational efficiency. ScaleX Innovation provides tailored solutions across multiple industry verticals to help businesses harness the power of AI-driven digital transformation. Their services include automating workflows, content analysis, and custom model implementations. They are known for their commitment to ethical compliance, versatility, and their role as a trusted partner for businesses worldwide.'

In [12]:
query = "What is the contact information?"
result = conversation_chain({"question": query})
answer = result["answer"]
answer

'You can contact ScaleX Innovation through the following means:\n\nAddress: Route Mahdia km 0.5, Pavillon d’Or Building, 3000 Sfax, Tunisia.\n\nEmail: info@scalexi.ai\n\nPhone Number (WhatsApp): +216-55-770-606\n\nFax: +216-55-770-606'